In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


## EDA of horse survival

## Questions

 - is train and original model in similar 1d distribution and 2d mutual info?
 - is there well defined margins for interpolation problem?
 - can we extrapolate?
 - where do measurement error come from?
 - are errors iid? are errors with finite mean, var, and is there any stochasicity?


In [ ]:
import pandas as pd
import numpy as np
sample_submission = pd.read_csv('/kaggle/input/playground-series-s3e22/sample_submission.csv')
train = pd.read_csv('/kaggle/input/playground-series-s3e22/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s3e22/test.csv')
original_df = pd.read_csv('/kaggle/input/horse-survival-dataset/horse.csv')

dfs = pd.concat([train, original_df]).drop_duplicates()



print(dfs.shape)

ids, X, y = dfs['id'], dfs.drop(['id','outcome'], axis = 1), dfs['outcome']
dfs.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
def make_pairplot(df, target = 'outcome',
                 plot = False,
                 save = True,
                 path = "scatter.png"):



    # Separate the target variable (optional, adjust as needed)
    target = df[target]

    # Create a scatter matrix using Pandas
    scatter_matrix = pd.plotting.scatter_matrix(df, alpha=0.1, s=.1, c=pd.Categorical(target).codes, figsize=(8, 8), diagonal='hist')

    # Customize the plot labels, colors, and other settings
    for ax in scatter_matrix.ravel():
        ax.set_xlabel(ax.get_xlabel(), fontsize=12)
        ax.set_ylabel(ax.get_ylabel(), fontsize=12)
        ax.xaxis.label.set_rotation(45)
        ax.yaxis.label.set_rotation(0)
        ax.yaxis.labelpad = 20
        ax.grid(False)

    # Add a legend
    colors = ['blue', 'orange']
    labels = target.unique()
    legend_labels = {code: label for code, label in zip(pd.Categorical(target).codes, labels)}
    for code, label in legend_labels.items():
        scatter_matrix[0, 0].plot([], [], c=('red','blue','green','purple','orange')[code], label=label)

    # Show the legend
    scatter_matrix[0, 0].legend()

    # Save the scatter matrix to a file (e.g., PNG)
    plt.savefig(path)

    # Show the scatter matrix (optional)
    plt.show()


In [ ]:
# make_pairplot(train, path="train.png")
# make_pairplot(original_df, path="original.png")

In [ ]:
train.select_dtypes(include=['number']).columns

In [ ]:
original_df.columns

In [ ]:
def create_pair_plot_comparision(df, title, target = "outcome"):
    num_features = df.select_dtypes(include=['number']).columns
    num_features_count = len(num_features)

    fig, axes = plt.subplots(nrows=num_features_count, ncols=num_features_count, figsize=(40,40))
    fig.suptitle(title)

    for i, feature1 in enumerate(num_features):
        for j, feature2 in enumerate(num_features):
            if i == j:
                axes[i, j].hist(df[feature1], bins=10, alpha=0.7)
#                 axes[i, j].set_title(feature1)
            else:
                axes[i, j].scatter(df[feature2], df[feature1], c=df[target].astype('category').cat.codes, cmap='Set1', alpha=0.7)
                
            if i==num_features_count-1:
                axes[i, j].set_xlabel(feature2)
            if j==0:
                axes[i, j].set_ylabel(feature1)


    plt.tight_layout(rect=[0, 0, 1, 0.95])  # Adjust the layout
    plt.savefig('pairplots.png')
create_pair_plot_comparision(train.drop('id', axis=1), "Comparing train df and original dataset")

In [ ]:
def create_pair_plot_comparision(df, df2, title):
    num_features = df.select_dtypes(include=['number']).columns
    num_features_count = len(num_features)

    fig, axes = plt.subplots(nrows=num_features_count, ncols=num_features_count, figsize=(40,40))
    fig.suptitle(title)

    for i, feature1 in enumerate(num_features):
        for j, feature2 in enumerate(num_features):
            if i == j:
                axes[i, j].hist(df[feature1], bins=10, alpha=0.2,
                               color = 'red', edgecolor = 'white')
#                 axes[i, j].set_title(feature1)
            else:
                axes[i, j].scatter(df[feature2], df[feature1], c='red',alpha = .3)
                
            if i==num_features_count-1:
                axes[i, j].set_xlabel(feature2)
            if j==0:
                axes[i, j].set_ylabel(feature1)

    for i, feature1 in enumerate(num_features):
        for j, feature2 in enumerate(num_features):
            if i == j:
                axes[i, j].hist(df2[feature1], bins=10, alpha=0.2,
                                                  color = 'blue', edgecolor = 'black')
#                 axes[i,num_features_count+ j].set_title(feature1)
            else:
                axes[i, j].scatter(df2[feature2], df2[feature1], c='blue', alpha=0.3)

    plt.tight_layout(rect=[0, 0, 1, 0.95])  # Adjust the layout
    plt.savefig('pairplots_inplace_comparison.png')


create_pair_plot_comparision(dfs.drop('outcome',axis=1),test, "Train test comparison")

In [ ]:
# df1 = train
df2 = original_df

# Function to create pair plots without Seaborn
def create_pair_plot_comparision(df, df2, title, target = "outcome"):
    num_features = df.select_dtypes(include=['number']).columns
    num_features_count = len(num_features)

    fig, axes = plt.subplots(nrows=num_features_count, ncols=num_features_count*2, figsize=(80,40))
    fig.suptitle(title)

    for i, feature1 in enumerate(num_features):
        for j, feature2 in enumerate(num_features):
            if i == j:
                axes[i, j].hist(df[feature1], bins=10, alpha=0.7)
#                 axes[i, j].set_title(feature1)
            else:
                axes[i, j].scatter(df[feature2], df[feature1], c=df[target].astype('category').cat.codes, cmap='Set1', alpha=0.7)
                
            if i==num_features_count-1:
                axes[i, j].set_xlabel(feature2)
            if j==0:
                axes[i, j].set_ylabel(feature1)
    
    for i, feature1 in enumerate(num_features):
        for j, feature2 in enumerate(num_features):
            if i == j:
                axes[i,num_features_count+ j].hist(df2[feature1], bins=10, alpha=0.7)
#                 axes[i,num_features_count+ j].set_title(feature1)
            else:
                axes[i, num_features_count+ j].scatter(df2[feature2], df2[feature1], c=df2[target].astype('category').cat.codes, cmap='Set1', alpha=0.7)
            if i == num_features_count-1:
                axes[i, num_features_count+ j].set_xlabel(feature2)


    plt.tight_layout(rect=[0, 0, 1, 0.95])  # Adjust the layout
    plt.savefig('pairplots_combined.png')

    
create_pair_plot_comparision(train.drop('id', axis=1), original_df, "Comparing train df and original dataset")

In [ ]:
import pandas as pd
from sklearn.feature_selection import mutual_info_regression, mutual_info_classif

# All numeric columns
features = dfs.select_dtypes(include=['number']).columns

# Number of bins for discretization
num_bins = 10

mi_matrix = pd.DataFrame(index=features, columns=features, dtype=float)

def bin_continuous_data(data):
    """Bin continuous data."""
    data_binned, _ = pd.cut(data, bins=num_bins, labels=range(num_bins), retbins=True)
    return data_binned

def is_continuous(data, threshold):
    return data.nunique() > threshold

for i, fi in enumerate(features):
    for j, fj in enumerate(features):
        if i == j:
            mi_matrix.iloc[i, j] = 1.0  # Mutual information of a feature with itself is 1.0
        else:
            dfs_local = dfs[[fi, fj]].dropna()
            
            if is_continuous(dfs_local[fi], num_bins) and is_continuous(dfs_local[fj], num_bins):
                # Both are continuous
                X = dfs_local[fi].values.reshape(-1, 1)
                y = dfs_local[fj]
                mi = mutual_info_regression(X, y)[0]
            elif not is_continuous(dfs_local[fi], num_bins) and not is_continuous(dfs_local[fj], num_bins):
                # Both are discrete
                mi = mutual_info_classif(dfs_local[fi].values.reshape(-1, 1), dfs_local[fj])[0]
            else:
                # One is continuous and the other is discrete
                if is_continuous(dfs_local[fi], num_bins):
                    dfs_local[fi] = bin_continuous_data(dfs_local[fi])
                else:
                    dfs_local[fj] = bin_continuous_data(dfs_local[fj])
                mi = mutual_info_classif(dfs_local[fi].values.reshape(-1, 1), dfs_local[fj])[0]
            
            mi_matrix.iloc[i, j] = mi

# Display the mutual information matrix
print(mi_matrix)


In [ ]:
import seaborn as sns
plt.figure(figsize=(10, 6))
sns.heatmap(np.array(mi_matrix), annot=True, cmap='coolwarm', fmt='.2f', square=True,
                        xticklabels=mi_matrix.columns, yticklabels=mi_matrix.columns,
            annot_kws={'fontsize': 5}) 

# Customize the plot
plt.title('Pairwise Mutual Information Heatmap')